In [1]:
import pandas as pd
import itertools as it
import gurobipy as gp
from gurobipy import GRB

In [2]:
data1 = [[0, 20, 25, 35, 65, 90, 85, 80, 86, 25, 35, 20, 44, 35, 82],
[20, 0, 15, 35, 60, 55, 57, 85, 90, 25, 35, 30, 37, 20, 40],
[25, 15, 0, 30, 50, 70, 55, 50, 65, 10, 25, 15, 24, 20, 90],
[35, 35, 30, 0, 45, 60, 53, 55, 47, 12, 22, 20, 12, 10, 21],
[65, 60, 50, 45, 0, 46, 15, 45, 75, 25, 11, 19, 15, 25, 25],
[90, 55, 70, 60, 46, 0, 15, 15, 25, 45, 65, 53, 43, 63, 70],
[85, 57, 55, 53, 15, 15, 0, 17, 25, 41, 25, 33, 27, 45, 30],
[80, 85, 50, 55, 45, 15, 17, 0, 25, 40, 34, 32, 20, 30, 10],
[86, 90, 65, 47, 75, 25, 25, 25, 0, 65, 70, 72, 61, 45, 13],
[25, 25, 10, 12, 25, 45, 41, 40, 65, 0, 20, 8, 7, 15, 25],
[35, 35, 25, 22, 11, 65, 25, 34, 70, 20, 0, 5, 12, 45, 65],
[20, 30, 15, 20, 19, 53, 33, 32, 72, 8, 5, 0, 14, 34, 56],
[44, 37, 24, 12, 15, 43, 27, 20, 61, 7, 12, 14, 0, 30, 40],
[35, 20, 20, 10, 25, 63, 45, 30, 45, 15, 45, 34, 30, 0, 27],
[82, 40, 90, 21, 25, 70, 30, 10, 13, 25, 65, 56, 40, 27, 0]]

cities = ['Heathrow', 'Harrow', 'Ealing', 'Holborn', 'Sutton', 'Dartford', 'Bromley', 'Greenwich', 'Barking', 'Hammersmith', 'Kingston', 'Richmond', 'Battersea', 'Islington', 'Woolwich']
depot = 'Heathrow'
dist_m = pd.DataFrame(data1, columns=cities, index=cities)
t_limt = 120
vans = list(range(1, 7))

ijk = list(it.product(it.permutations(cities,2), vans))
ij = list(it.permutations(cities,2))
ik = list(it.product(cities, vans))

In [3]:
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._x)        
        #for k in vans:
        #    for i,j in ij:
        #        if vals[(i, j), k] > 0.5:
        selected = gp.tuplelist((e[0], e[1]) for e,k in ijk if vals[(e[0], e[1]), k] > 0.5)
                    
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < len(cities): 
            for k in vans:
                model.cbLazy(gp.quicksum(model._x[(i, j), k]
                                         for i, j in it.permutations(tour, 2))
                             <= len(tour)-1)

# Given a tuplelist of edges, find the shortest subtour not containing depot
def subtour(edges):
    unvisited = [c for c in cities[1:]]      
    cycle = range(len(cities)+1)  # initial length has 1 more city
    
    # First, remove all nodes connected to depot
    depot_connected = [j for i, j in edges.select(depot, '*')]
    while depot_connected:
        current = depot_connected.pop()
        unvisited.remove(current)
        neighbors = [j for i, j in edges.select(current, '*')
                     if j in unvisited and j != depot]
        depot_connected += neighbors

    # Now, find subtour using tsp.py code
    while unvisited:
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

In [7]:
model = gp.Model('Lost Baggage')

# add vars
x = model.addVars(ijk, vtype=GRB.BINARY,
                  name='x')
y = model.addVars(ik, vtype=GRB.BINARY,
                  name='y')
s = model.addVars(vans, vtype=GRB.BINARY,
                  name='s')

# add objective function
model.setObjective(s.sum())

# add constraints

    # if van visits a location, than van is used
model.addConstrs((y[i,k] <= s[k] for k in vans for i in cities),
                 name='used_van')

    # each location is visited by only one van
model.addConstrs((gp.quicksum(y[i,k] for k in vans) == 1 for i in cities if i != depot),
                 name='visit')

    # all vans visit Heathrow
model.addConstr((gp.quicksum(y[depot,k] for k in vans)
                 >= gp.quicksum(s[k] for k in vans)),
                name='visitDepot')

    # connected routes
for j,k in ik:
    model.addConstr((gp.quicksum(x[(i,j),k] for i in cities if i != j)
                     == y[j,k]))    
for j,k in ik:
    model.addConstr((gp.quicksum(x[(j,i),k] for i in cities if i != j)
                     == y[j,k]))
    
    # avoid symmetric solutions
for k in vans[:5]:
    model.addConstr((gp.quicksum(y[i,k] for i in cities)
                     >= gp.quicksum(y[i,k+1] for i in cities)),
                    name='symm'+str(k))
    
    # route time limit
model.addConstrs((gp.quicksum(dist_m[i][j]*x[(i,j),k] for i,j in ij if j != 'Heathrow')
                  <= t_limt for k in vans),
                 name='t_limit')    
model.update()
model.write('Lost Baggage.lp')

model._x = x                      # that's how we pass data from the model to the callback function, using _
model.Params.LazyConstraints = 1  # set the parameter to use lazy constraints
model.optimize(subtourelim)       # indicate the function to callback here

Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 296 rows, 1356 columns and 4302 nonzeros
Model fingerprint: 0x82e747b9
Variable types: 0 continuous, 1356 integer (1356 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve time: 0.01s
Presolved: 296 rows, 1356 columns, 4302 nonzeros
Variable types: 0 continuous, 1356 integer (1356 binary)

Root relaxation: objective 1.000000e+00, 455 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.28662    0   69          -    1.28662      -     -    0s
     0     0    2.00000    0   99          - 

In [5]:
routes = {}
route_t = {}
for k in vans:
    if s[k].x>0:
        routes[k] = []        
        route_time = 0
        for i,j in ij:
            if x[(i,j),k].x > 0:
                route_time += dist_m[i][j]
                routes[k].append((i,j))                
        route_t[k] = route_time

In [6]:
for k in route_t.keys():
    print('Van {} route. Route total time: {}min'.format(k, route_t[k]))
    curr_city = routes[k][0][0]
    print(curr_city)
    for _ in range(len(routes[k])):
        next_trip = [(i,j) for i,j in routes[k] if i == curr_city]
        print(next_trip[0][1])
        curr_city = next_trip[0][1]
    print()

Van 1 route. Route total time: 182min
Heathrow
Harrow
Ealing
Hammersmith
Islington
Holborn
Battersea
Greenwich
Heathrow

Van 2 route. Route total time: 209min
Heathrow
Richmond
Kingston
Sutton
Bromley
Woolwich
Barking
Dartford
Heathrow

